# HW 2 Multimodal Machine Learning for Emotion Recognition

- main with sub notebooks
    1. audio (acoustic) 
    2. text (lexical) 
    3. visual (this notebook)
    
- `%load` and `%run` with [cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-load): This allows us to get code from another notebook

In [1]:
main_notebook = 'main.ipynb'

In [2]:
# %load main_notebook
main.ipynb

In [3]:
%run 'main.ipynb'

- If video, the original video will be a 3D with a T dimension. This specific case, we have a 2D of (T, 2048) which is based on the ResNet encoder

In [4]:
# features_with_y

In [5]:
ys = features_with_y.loc[0:, 'emotion_labels']
# ys

In [6]:
def reduce_temporal_dimension(visual_features_paths, ys):
    """Reduce from 2D to 1D by removing the time dimension
    
    visual_features_paths -- list
    
    Return 
    reduced shapes of each input -- list
    """
    reduced_visual_features = []
    true_labels = []
    
    for row in range(len(visual_features_paths)):
        # print(ys[row])
        # print("Current path with files is: ", visual_features_path)
        path_exists = os.path.exists(visual_features_paths[row])
        # print(path_exists)
        if path_exists == True:
            # print("Current path with files is: ", visual_features_path)
            load_visual_features_file = np.load(visual_features_paths[row])
            # print("  Original Shape: ", np.shape(load_visual_features_file))
            resampled_visual = np.mean(load_visual_features_file, axis=0)
            reduced_visual_features.append(resampled_visual)
            # print("  Reduced shape: ", np.shape(resampled_visual))
            # print()
            true_labels.append(ys[row])
        else:
            # print()
            
            # print("\nCANNOT find current path: ", visual_features_paths[row])
            # print("DoNOT include", ys[row])
            pass
    return reduced_visual_features, true_labels

In [20]:
reduced_temporal_dimensions, true_labels = reduce_temporal_dimension(visual_features_paths, ys)
np.shape(reduced_temporal_dimensions), np.shape(true_labels)

((1003, 2048), (1003,))

See my [HW 2 Notes](https://detraviousjbrinkley.notion.site/HW-2-Multimodal-Machine-Learning-for-Emotion-Recognition-4967e03039b04f20907996de25f6068a) for more details on 
1. Support Vector Machine (SVM)
2. Support Vector Classification (SVC)
3. Note that LinearSVC does not accept parameter kernel, as this is assumed to be linear per the [SVM documentation](https://scikit-learn.org/stable/modules/svm.html#multi-class-classification)

In [25]:
linear_classifier = svm.LinearSVC()
parameters = {'SVC__C':np.arange(0.01,100,10)}
classification = GridSearchCV(linear_classifier, param_grid=parameters, cv=10)
classification

GridSearchCV(cv=10, estimator=LinearSVC(),
             param_grid={'SVC__C': array([1.000e-02, 1.001e+01, 2.001e+01, 3.001e+01, 4.001e+01, 5.001e+01,
       6.001e+01, 7.001e+01, 8.001e+01, 9.001e+01])})

In [27]:
# classification.fit(reduced_temporal_dimensions, true_labels)